In [2]:
import cv2
import time

# Load the pre-trained face detection model (Haar Cascade Classifier)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the default camera (usually the built-in webcam)
cap = cv2.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Initialize variables for FPS calculation
start_time = time.time()
frame_count = 0

# Initialize variables for face movement tracking
prev_faces = None

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        print("Error: Could not read frame.")
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform face detection
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Calculate the center points of the detected face
        current_face_center = (x + w // 2, y + h // 2)

        # Draw a rectangle around the detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Calculate movement if there is a previous face position
        if prev_faces is not None:
            prev_face_center = prev_faces[0]

            # Calculate the distance between current and previous face centers
            distance = int(((current_face_center[0] - prev_face_center[0])**2 +
                            (current_face_center[1] - prev_face_center[1])**2)**0.5)

            # Display the movement distance inside the rectangle
            cv2.putText(frame, f"Movement: {distance}", (x + 10, y + h + 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Define points and descriptions
        points = [current_face_center, (x + w // 2, y + h // 2 + h // 6),
                  (x + w // 4, y + h // 2 + h // 4), (x + 3 * w // 4, y + h // 2 + h // 4)]
        descriptions = ["Eye Center", "Nose Tip", "Mouth Left Corner", "Mouth Right Corner"]

        # Draw dots (circles) at the calculated points
        for point, description in zip(points, descriptions):
            cv2.circle(frame, point, 4, (255, 0, 0), -1)  # Blue dot
            cv2.putText(frame, description, (point[0] + 10, point[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Update the previous faces for the next iteration
        prev_faces = [current_face_center]

    # Calculate FPS
    frame_count += 1
    elapsed_time = time.time() - start_time
    fps = frame_count / elapsed_time

    # Display FPS on the frame
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame with face detection, dots, and movement measurement
    cv2.imshow('Face Recognition', frame)

    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
